<a href="https://colab.research.google.com/github/CllsPy/Gen-AI/blob/main/Gemma_on_Colabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Question answering

1. [Model Card for C4AI Command R+]( https://huggingface.co/CohereForAI/c4ai-command-r-plus)
2. [Authentication](https://huggingface.co/docs/hub/security-tokens)
3. [What is good model for production - quantized vs non-quantized](https://www.reddit.com/r/LocalLLaMA/comments/15g6ds4/what_is_good_model_for_production_quantized_vs/?rdt=51663&onetap_auto=true&one_tap=true)
4. [Gemma on Colabs](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/pytorch_gemma.ipynb#scrollTo=0qu4_r3PycgW)
5. [Gemma Medium](https://medium.com/@coldstart_coder/getting-started-with-googles-gemma-llm-using-huggingface-libraries-a0d826c552ae)

## Kaggle Acess

In [ ]:
import kagglehub

kagglehub.login()

## Install dependencies

In [ ]:
!pip install -q -U torch immutabledict sentencepiece

## Download Model

In [ ]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cpu' #@param ['cuda', 'cpu']

In [ ]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [ ]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

In [ ]:
import sys

sys.path.append('gemma_pytorch')

In [ ]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [ ]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [ ]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='Onde viajar se eu estiver no Brasil?' # <- Pergunta
    )
    #+ MODEL_CHAT_TEMPLATE.format(prompt='California.')
    #+ USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    #+ '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=100,
)

## Resposta

Aonde viajar na Brasil depende do seu destino.

* **No interior do Brasil:**
    * Aviação é a forma mais rápida e eficiente de chegar ao interior.
    * Existem várias empresas de ônibus de passageiro, como ViaLiner e Gaivão, que oferecem viagensBegriffskläros entre cidades e cidades dentro do país.
    * A estrada é uma opção, mas é importante levar em conta as condições meteorológicas e a segurança na estrada.

*